In [12]:
from sklearn.model_selection import train_test_split
import numpy as np

Научимся считывать данные. В качестве человеческого текста будем использовать строки из книг, а в качестве кода - код решений для задач по курсу алгоритмам за прошлый семестр.
Всего строк текста около 8к, кода даже больше, чем нужно, поэтому возьмем, чтобы его было 20%

In [13]:
def read_text(path: str, mode='book'):
    """
    Для чтения возможно 2 случая - для книг и для кода
    В случае чтения файла с кодом, дополнительно убираются все комментарии и отступы в начале строки
    """
    X = []
    with open(path, 'r') as text:
        flag = True
        for s in text:
            line = ''
            if s!='\n':
                line = s.lower().replace('\n', '').replace('\t', '')
            if mode=='code':
                line = ' '.join(map(str, line.split()))
                if '#' in line:
                    line = line[:line.find('#')]
                if "'''" in line:
                    flag = not flag
            if flag and "'''" not in line and line != '':
                X.append(line)
    return X

In [14]:
from os import listdir
from os.path import join, isdir

def read_book(mypath: str):
    """
    Обход .txt файлов
    """
    X = []
    for f in listdir(mypath):
        X.extend(read_text(join(mypath, f)))
    return X

def read_code(mypath: str):
    """
    Обход .py файлов в папочках
    """
    X = []
    # for d in listdir(mypath):
    #     new_path = join(mypath, d)
    #     if isdir(new_path):
    #         for f in listdir(new_path):
    #             X.extend(read_text(join(new_path, f), mode='code'))
    for f in listdir(mypath):
        X.extend(read_text(join(mypath, f), mode='code'))
    return X

In [15]:
def read_dataset(book_path: str, code_path: str):
    """
    Собираем датасет
    0 - человеческий текст, 1 - код
    """
    X1 = read_book(book_path)
    X2 = read_code(code_path)[:int(len(X1) * 0.16)]
    # print(X2[:100])
    print(f'books {len(X1)}\ncode {len(X2)}')
    y1 = np.zeros(len(X1))
    y2 = np.ones(len(X2))
    return np.append(X1, X2), np.append(y1, y2)

In [16]:
X, y = read_dataset('./books', './tasks')
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

books 8356
code 1336


Чтобы кодировать строки текста, будем использовать простейший мешок слов. Реализуем сами (это моя реализация из соответсвующей домашки по ML).

In [17]:
class BoW:
    def __init__(self, X: np.ndarray, voc_limit: int = 1000):
        """
        Составляет словарь, который будет использоваться для векторизации предложений.

        Parameters
        ----------
        X : np.ndarray
            Массив строк (предложений) размерности (n_sentences, ),
            по которому будет составляться словарь.
        voc_limit : int
            Максимальное число слов в словаре.

        """
        self.vocab_size = None
        self.d = {}
        for sentence in X:
            s = self.remaker(sentence)
            for word in s:
                if word in self.d:
                    self.d[word] += 1
                else:
                    self.d[word] = 1
        all_words = np.array(sorted(list(self.d.items()),key=lambda x: -x[1]))
        self.d = all_words[:voc_limit, 0]     # убрать самые частые?
        self.vocab_size = self.d.shape[0]
        self.d = dict(np.concatenate((self.d[:, None], np.arange(len(self.d))[:, None]), axis=1))


    def remaker(self, sentence):
        simb = [',',"'", '!', '.', ',', '?', '*']
        for x in simb:
            sentence = sentence.replace(x, ' ')
        return sentence.lower().split()

    def transform(self, X: np.ndarray) -> np.ndarray:
        """
        Векторизует предложения.

        Parameters
        ----------
        X : np.ndarray
            Массив строк (предложений) размерности (n_sentences, ),
            который необходимо векторизовать.

        Return
        ------
        np.ndarray
            Матрица векторизованных предложений размерности (n_sentences, vocab_size)
        """
        res = np.zeros((X.shape[0], self.vocab_size))
        for i in range(X.shape[0]):
            s = self.remaker(X[i])
            for word in s:
                if word in self.d:
                    j = int(self.d[word])
                    res[i, j] += 1
        return res


В качестве модели воспользуемся наивным байесом
Реализуем его сами

In [18]:
class NaiveBayes:
    def __init__(self, alpha: float):
        """
        Parameters
        ----------
        alpha : float
            Параметр аддитивной регуляризации.
        """
        self.alpha = alpha
        self.classes = None
        self.f_probas = None
        self.n_classes = None
        self.y_probas = None

    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        Оценивает параметры распределения p(x|y) для каждого y.
        """
        self.classes, counts = np.unique(y, return_counts=True)
        self.n_classes = len(self.classes)
        n_dict = X.shape[1]
        self.f_probas = np.zeros((n_dict, self.n_classes))
        self.y_probas = counts / np.sum(counts)

        for j in range(self.n_classes):
            j_class = y== self.classes[j]
            x_i_j_class = np.sum(X[j_class], axis=0)
            all_j_class = np.sum(X[j_class])
            self.f_probas[:, j] = (x_i_j_class + self.alpha) / (all_j_class + self.n_classes * self.alpha)


    def predict(self, X: np.ndarray):
        """
        Return
        ------
        list
            Предсказанный класс для каждого элемента из набора X.
        """
        return np.array([self.classes[i] for i in np.argmax(self.log_proba(X), axis=1)])

    def log_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Return
        ------
        np.ndarray
            Для каждого элемента набора X - логарифм вероятности отнести его к каждому классу.
            Матрица размера (X.shape[0], n_classes)
        """
        res = np.zeros((X.shape[0], self.n_classes))
        for i in range(X.shape[0]):
            for j in range(self.n_classes):
                res[i, j] = np.sum(X[i, :] * np.log(self.f_probas[:, j])) + self.y_probas[j]
        return res

Научимся считать метрики

In [19]:
def get_precision_recall_accuracy(y_pred, y_true):
    true = sum(y_pred == y_true)
    # все правильные делить на все неправильные - accuracy
    all = y_pred.shape[0]
    accuracy = true / all
    precision, recall = np.array([]), np.array([])
    classes = np.unique(list(y_pred) + list(y_true))
    for item in classes:
        TP = sum((y_pred == item) * (y_true == item))
        FP = sum((y_pred == item) * (y_true != item))
        FN = sum((y_pred != item) * (y_true == item))
        if TP + FP > 0:
            precision = np.append(precision, [TP / (TP + FP)])
        else:
            precision = np.append(precision, [0])
        if TP + FN > 0:
            recall = np.append(recall, [TP / (TP + FN)])
        else:
            recall = np.append(recall, [0])
    precision = dict((c, precision[i]) for i, c in enumerate(classes))
    recall = dict((c, recall[i]) for i, c in enumerate(classes))
    print(f'precision: {precision} \nrecall: {recall} \naccuracy: {accuracy}')

Обучим модель и посмотрим, что получается

In [20]:
bow = BoW(X_train, voc_limit=1000)
X_train_bow = bow.transform(X_train)
X_test_bow = bow.transform(X_test)

In [21]:
predictor = NaiveBayes(0.001)
predictor.fit(X_train_bow, y_train)
get_precision_recall_accuracy(predictor.predict(X_test_bow), y_test)

precision: {0.0: 0.9805309734513274, 1.0: 0.9877049180327869} 
recall: {0.0: 0.9981981981981982, 1.0: 0.8795620437956204} 
accuracy: 0.9814337287261475


In [22]:
x = np.array(["yield from range(number + 1, number + on_each_side + 1)",
              "If the page range is larger than a given size, the whole range is not"])
tr = bow.transform(np.array(x))
print(predictor.predict(tr))

[1. 0.]


Видно, что значения precision очень большие, значит, из того, что мы помечаем как нужный класс, мы очень часто угадываем
Что касается recall, то поскольку у нас данные неравномерные, а так же из-за специфики данных, в словарь попадает очень много человеческих слов и мало кода. Поэтому мы очень точно находим человеческий текст и часто ошибаемся в случаях, когда сталкиваемся с кодом.
На тех данных, на которых она обучилась, модель работает в общем не плохо.